In [2]:
import json

# Load the training challenges and solutions
with open('arc-agi_training_challenges.json', 'r') as file:
    training_challenges = json.load(file)

with open('arc-agi_training_solutions.json', 'r') as file:
    training_solutions = json.load(file)

# Load the evaluation challenges and solutions
with open('arc-agi_evaluation_challenges.json', 'r') as file:
    evaluation_challenges = json.load(file)

with open('arc-agi_evaluation_solutions.json', 'r') as file:
    evaluation_solutions = json.load(file)

In [3]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch

def pad_grid(grid, target_size=(30, 30)):
    padded_grid = np.zeros(target_size, dtype=int)
    padded_grid[:grid.shape[0], :grid.shape[1]] = grid
    return padded_grid

class ARCDataset(Dataset):
    def __init__(self, challenges, solutions):
        self.challenges = challenges
        self.solutions = solutions
        self.keys = list(challenges.keys())

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        key = self.keys[idx]
        challenge = self.challenges[key]
        solution = self.solutions[key]

        # Extracting and padding the train and test grids
        train_inputs = [pad_grid(np.array(pair['input'])) for pair in challenge['train']]
        train_outputs = [pad_grid(np.array(pair['output'])) for pair in challenge['train']]
        test_inputs = [pad_grid(np.array(pair['input'])) for pair in challenge['test']]
        test_outputs = [pad_grid(np.array(output)) for output in solution]

        # Convert to tensors
        train_inputs = torch.tensor(train_inputs, dtype=torch.float32)
        train_outputs = torch.tensor(train_outputs, dtype=torch.float32)
        test_inputs = torch.tensor(test_inputs, dtype=torch.float32)
        test_outputs = torch.tensor(test_outputs, dtype=torch.float32)

        # Ensure the inputs have the correct dimensions
        train_inputs = train_inputs.unsqueeze(1)
        train_outputs = train_outputs.unsqueeze(1)
        test_inputs = test_inputs.unsqueeze(1)
        test_outputs = test_outputs.unsqueeze(1)

        return train_inputs, train_outputs, test_inputs, test_outputs

# Create DataLoader for training data
train_dataset = ARCDataset(training_challenges, training_solutions)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

class ARCModel(nn.Module):
    def __init__(self):
        super(ARCModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 30 * 30, 512)
        self.fc2 = nn.Linear(512, 30 * 30)
        self.output_size = (30, 30)

    def forward(self, x):
        batch_size, num_pairs, _, height, width = x.size()
        x = x.view(batch_size * num_pairs, 1, height, width)
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = x.view(batch_size * num_pairs, -1)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        x = x.view(batch_size, num_pairs, *self.output_size)
        return x

# Initialize the model, loss function, and optimizer
model = ARCModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 12

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for train_inputs, train_outputs, _, _ in train_loader:
        optimizer.zero_grad()
        outputs = model(train_inputs)
        outputs = outputs.unsqueeze(2)
        loss = criterion(outputs, train_outputs)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")


<ipython-input-3-3c1bd280173f>:31: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  train_inputs = torch.tensor(train_inputs, dtype=torch.float32)


Epoch [1/12], Loss: 1.1450636206066702
Epoch [2/12], Loss: 1.0877350332168862
Epoch [3/12], Loss: 0.9561793418461457
Epoch [4/12], Loss: 0.9859869396244175
Epoch [5/12], Loss: 0.8384455952607095
Epoch [6/12], Loss: 0.8078570779412985
Epoch [7/12], Loss: 0.7188617905974388
Epoch [8/12], Loss: 0.7306903638038784
Epoch [9/12], Loss: 0.6928759471932426
Epoch [10/12], Loss: 0.632548714694567


KeyboardInterrupt: 

#testing


In [5]:
# Define the evaluation function
def evaluate_model(model, loader):
    model.eval()
    total_loss = 0.0
    criterion = nn.MSELoss()

    with torch.no_grad():
        for test_inputs, _, test_outputs, _ in loader:
            outputs = model(test_inputs)

            # Ensure the output size matches the target size
            outputs = outputs.unsqueeze(2)
            if outputs.size() != test_outputs.size():
                min_size = min(outputs.size(1), test_outputs.size(1))
                outputs = outputs[:, :min_size, :, :, :]
                test_outputs = test_outputs[:, :min_size, :, :, :]

            loss = criterion(outputs, test_outputs)
            total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    print(f"Evaluation Loss: {avg_loss}")
    return avg_loss

# Evaluate the model
evaluation_loss = evaluate_model(model, evaluation_loader)


Evaluation Loss: 2.386026121396571


In [6]:
# Define the comparison function
def compare_results(model, loader):
    model.eval()
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for test_inputs, _, test_outputs, _ in loader:
            outputs = model(test_inputs)
            outputs = outputs.unsqueeze(2)

            if outputs.size() != test_outputs.size():
                min_size = min(outputs.size(1), test_outputs.size(1))
                outputs = outputs[:, :min_size, :, :, :]
                test_outputs = test_outputs[:, :min_size, :, :, :]

            all_predictions.append(outputs.cpu().numpy())
            all_targets.append(test_outputs.cpu().numpy())

    # Compare predictions with targets
    for i in range(len(all_predictions)):
        print(f"Prediction {i+1}:")
        print(all_predictions[i])
        print(f"Target {i+1}:")
        print(all_targets[i])
        print("\n")

# Compare model predictions with ground truth solutions
compare_results(model, evaluation_loader)

Streaming output truncated to the last 5000 lines.
     -6.55232295e-02  3.02324831e-01  1.91739500e-01  1.78503674e-02
     -7.01985210e-02  8.46312195e-02  1.46295130e-02  1.66078493e-01
      1.12232864e-01  2.13962123e-02]
    [-1.18043691e-01 -2.57948011e-01 -2.33496130e-01 -3.04530829e-01
     -2.35525861e-01 -3.46179157e-02 -7.33244479e-01 -5.06184220e-01
     -2.03122646e-01 -1.78639412e-01 -1.69051230e-01 -2.34967649e-01
     -1.68949693e-01 -4.72021341e-01  8.86054933e-02 -4.52846944e-01
     -1.59853950e-01 -1.90887004e-02 -7.80185908e-02  1.53240517e-01
     -1.17454827e-01 -1.18828863e-01 -3.81450653e-02 -1.53595239e-01
     -1.01601660e-01 -1.13728799e-01  6.41709864e-02  9.09584761e-03
      4.41705436e-03 -6.13645017e-02]
    [-2.46389329e-01 -4.95724380e-02  1.29712820e-01  1.56960621e-01
     -8.81454349e-02 -6.39142394e-02  9.63228941e-03  4.62061763e-02
      7.83573091e-02  4.72045541e-02 -1.87878743e-01 -2.81143904e-01
     -1.00284420e-01 -3.27266037e-01 -1.03859

In [1]:
# Load test data
with open('/mnt/data/arc-agi_test-challenges.json') as f:
    test_challenges = json.load(f)

# Create DataLoader for test data
test_dataset = ARCDataset(test_challenges, test_challenges)  # Using the same challenges as solutions for testing
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Predict function for test data
def predict(model, loader):
    model.eval()
    predictions = {}

    with torch.no_grad():
        for idx, (test_inputs, _, _, _) in enumerate(loader):
            outputs = model(test_inputs)
            predictions[f"test_case_{idx+1}"] = outputs.squeeze().cpu().numpy().tolist()

    return predictions

# Get predictions
test_predictions = predict(model, test_loader)

# Save predictions to JSON
with open('arc-agi_test_predictions.json', 'w') as f:
    json.dump(test_predictions, f)

print("Test predictions saved to 'arc-agi_test_predictions.json'")


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/arc-agi_test-challenges.json'